<a href="https://colab.research.google.com/github/Shreyasvaidya/Open-CV-Summer-Challenge/blob/main/Evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from torchvision import models
from torch import nn
from tqdm import tqdm
import torch
from torchvision import transforms
from PIL import Image

In [2]:
pretrainednet = models.mobilenet_v2()

pretrainednet

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [3]:
from google.colab import drive
drive.mount('/content/drive',force_remount = True)

Mounted at /content/drive


In [6]:
class Network(nn.Module):
    def __init__(self,pretrain = pretrainednet):
        super(Network,self).__init__()
        self.pretrained = pretrain
        self.tail = nn.Linear(1000,1352)
    def forward(self,*args):
        ans = []
        for arg in args:
            ans.append(self.tail(self.pretrained(arg)))
        return ans

In [7]:
model = Network()
model.load_state_dict(torch.load("/content/drive/MyDrive/ncv/modelmobile.pth"))

<All keys matched successfully>

In [ ]:
del pretrainednet

In [8]:
!unzip /content/drive/MyDrive/dataset.zip -d /content

Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset/train/M887/A1.jpg  
  inflating: /content/dataset/train/M887/A2.jpg  
  inflating: /content/dataset/train/M887/B0.jpg  
  inflating: /content/dataset/train/M873/A0.jpg  
  inflating: /content/dataset/train/M873/A1.jpg  
  inflating: /content/dataset/train/M873/A2.jpg  
  inflating: /content/dataset/train/M873/B5.jpg  
  inflating: /content/dataset/train/M873/B4.jpg  
  inflating: /content/dataset/train/M873/B3.jpg  
  inflating: /content/dataset/train/M873/B2.jpg  
  inflating: /content/dataset/train/M873/B1.jpg  
  inflating: /content/dataset/train/P1524/A3.jpg  
  inflating: /content/dataset/train/P1524/A2.jpg  
  inflating: /content/dataset/train/P1524/A4.jpg  
  inflating: /content/dataset/train/P1524/B5.jpg  
  inflating: /content/dataset/train/P1524/B4.jpg  
  inflating: /content/dataset/train/P1524/B6.jpg  
  inflating: /content/dataset/train/P1524/B3.jpg  
  inflating: /content/dataset/train/P1524/

In [9]:
import os
#creating list of image pairs with combinations and labels of 1 or 0
path = '/content/dataset/train/'
img_label_list = []

count = 0
authors = os.listdir(path)
for i,author_name in tqdm(enumerate(authors)):
  other_list = authors[i:i+2] if i<len(authors)-1 else authors[i:]
  for author_name2 in other_list:
    label = 1 if author_name == author_name2 else 0
    for img1 in os.listdir(os.path.join(path,author_name)):
      for img2 in os.listdir(os.path.join(path,author_name2)):
        if(author_name != author_name2 or img1 !=img2):

          img_label_list.append([os.path.join(path,author_name,img1),os.path.join(path,author_name2,img2),label])


1352it [00:01, 1298.21it/s]


In [10]:
len(img_label_list)

181198

In [11]:
class SiamesepairwiseDataset(torch.utils.data.Dataset):
  def __init__(self,array,path= None,transform = None):
    self.array =array
    self.transform = transform
    self.path = path

  def __getitem__(self,index):

    indexlist = self.array[index]

    if self.path is None:
      img0 = Image.open(indexlist[0])
      img0 = img0.convert("RGB")

      img1 = Image.open(indexlist[1])
      img1 = img1.convert("RGB")



    else:
      img0 = Image.open(os.path.join(self.path,indexlist[0]))
      img0 = img0.convert("RGB")

      img1 = Image.open(os.path.join(self.path,indexlist[1]))
      img1 = img1.convert("RGB")


    label = indexlist[2]

    if self.transform is not None:

      img0 = self.transform(img0)
      img1 = self.transform(img1)





    return img0,img1,label

  def __len__(self):
    return len(self.array)

In [12]:
mytraintransform = transforms.Compose([transforms.Resize((256,256)),transforms.CenterCrop(224) ,transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])]
                               )
myvaltransform = transforms.Compose([transforms.Resize((256,256)),transforms.CenterCrop(224) ,transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])]
                               )

to_tensor = transforms.Compose([transforms.ToTensor(),
                               ])

In [14]:
import random
import numpy as np
from torch.utils.data import Dataset,DataLoader
random.shuffle(img_label_list)
train_knn_ds =DataLoader(SiamesepairwiseDataset(np.array(img_label_list[:10586*4]),transform = myvaltransform),batch_size = 512)

In [15]:
batch_size = 512
import numpy as np
mlen = 10586*4
traindist = np.zeros(mlen)
trainl = np.zeros(mlen)
model.cuda()
model.eval()
counter = 0
with torch.no_grad():
  for i in tqdm(train_knn_ds):
    img0,img1,label = i
    img0,img1 = img0.cuda(),img1.cuda()
    [output1,output2] = model(img0,img1)
    diff = output1-output2
    dist_sq = torch.sum(torch.pow(diff, 2), 1)
    dist = torch.sqrt(dist_sq)
    if(counter+batch_size-1<mlen):
      traindist[counter:counter+batch_size] = dist.cpu()
      trainl[counter:counter+batch_size] = label
      counter+=batch_size
    else:
      traindist[counter:] = dist.cpu()
      trainl[counter:] = label
      counter+=batch_size





100%|██████████| 83/83 [15:58<00:00, 11.55s/it]


In [ ]:
import pickle
with open("traindist.pickle","wb")as file:
  pickle.dump([traindist,trainl],file)

In [16]:
import pandas as pd

valdf = pd.read_csv("/content/dataset/val.csv")
valdf

,img1_name,img2_name,label
0,70a4ba9a.jpg,58f68a00.jpg,1
1,e15b612e.jpg,16ce5df2.jpg,0
2,6c64d988.jpg,735d3636.jpg,1
3,b74681a6.jpg,cb50496d.jpg,0
4,c97aa428.jpg,6f9dc747.jpg,1
...,...,...,...
5765,4e7762f7.jpg,2f3d2bce.jpg,0
5766,c1c49f87.jpg,1072597f.jpg,0
5767,03e66099.jpg,f877b1b6.jpg,1
5768,8e056e51.jpg,f68e79a1.jpg,1


In [17]:
valdl =DataLoader(SiamesepairwiseDataset(np.array(valdf),transform = myvaltransform,path = '/content/dataset/val'),batch_size = 512)

In [18]:
batch_size = 512
import numpy as np
mlen = 5770
valdist = np.zeros(mlen)
val_l = np.zeros(mlen)
model.cuda()
model.eval()
counter = 0
with torch.no_grad():
  for i in tqdm(valdl):
    img0,img1,label = i
    img0,img1 = img0.cuda(),img1.cuda()
    [output1,output2] = model(img0,img1)
    diff = output1-output2
    dist_sq = torch.sum(torch.pow(diff, 2), 1)
    dist = torch.sqrt(dist_sq)
    if(counter+batch_size-1<mlen):
      valdist[counter:counter+batch_size] = dist.cpu()
      val_l[counter:counter+batch_size] = label
      counter+=batch_size
    else:
      valdist[counter:] = dist.cpu()
      val_l[counter:] = label
      counter+=batch_size





100%|██████████| 12/12 [02:08<00:00, 10.67s/it]


In [19]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
knn = KNeighborsClassifier()
traindist = traindist.reshape(-1,1)
valdist = valdist.reshape(-1,1)
knn.fit(traindist,trainl)
proba = knn.predict_proba(valdist).T[1].reshape(-1)
roc_auc_score(val_l,proba)

0.6088390017090731

In [20]:
df = pd.read_csv("/content/drive/MyDrive/ncv/test.csv")
df

,img1_name,img2_name
0,3861e092.jpg,b642bddc.jpg
1,776e0055.jpg,2d4e7d20.jpg
2,4bb7f909.jpg,af1b0fbf.jpg
3,0abee1d8.jpg,1c2641c9.jpg
4,1e18f51e.jpg,9fcf5c47.jpg
...,...,...
10581,c958571b.jpg,2df4a5a6.jpg
10582,6493f6be.jpg,33faf06a.jpg
10583,4dc5259f.jpg,0e85870e.jpg
10584,8771a705.jpg,e41f0ede.jpg


In [21]:
class SiamesetestDataset(torch.utils.data.Dataset):
  def __init__(self,array,path= None,transform = None):
    self.array =array
    self.transform = transform
    self.path = path

  def __getitem__(self,index):

    indexlist = self.array[index]

    if self.path is None:
      img0 = Image.open(indexlist[0])
      img0 = img0.convert("RGB")

      img1 = Image.open(indexlist[1])
      img1 = img1.convert("RGB")



    else:
      img0 = Image.open(os.path.join(self.path,indexlist[0]))
      img0 = img0.convert("RGB")

      img1 = Image.open(os.path.join(self.path,indexlist[1]))
      img1 = img1.convert("RGB")




    if self.transform is not None:

      img0 = self.transform(img0)
      img1 = self.transform(img1)





    return img0,img1

  def __len__(self):
    return len(self.array)

In [22]:
!unzip /content/drive/MyDrive/ncv/semi_test.zip

Archive:  /content/drive/MyDrive/ncv/semi_test.zip
  inflating: semi_test/025e5d31.jpg  
  inflating: semi_test/f2bb12b7.jpg  
  inflating: semi_test/af9f1abe.jpg  
  inflating: semi_test/a4589597.jpg  
  inflating: semi_test/320f6bf9.jpg  
  inflating: semi_test/9d22967f.jpg  
  inflating: semi_test/2323b00c.jpg  
  inflating: semi_test/1d320a7d.jpg  
  inflating: semi_test/dc2f775b.jpg  
  inflating: semi_test/7f8d59e0.jpg  
  inflating: semi_test/0f2bb7bc.jpg  
  inflating: semi_test/ea5005a2.jpg  
  inflating: semi_test/7e02401e.jpg  
  inflating: semi_test/986c5823.jpg  
  inflating: semi_test/ef0814a5.jpg  
  inflating: semi_test/f0ffd208.jpg  
  inflating: semi_test/aa1dd1f1.jpg  
  inflating: semi_test/3f602015.jpg  
  inflating: semi_test/595c2f80.jpg  
  inflating: semi_test/d69cea49.jpg  
  inflating: semi_test/fc22fe8d.jpg  
  inflating: semi_test/51b94946.jpg  
  inflating: semi_test/095aa184.jpg  
  inflating: semi_test/a29e58b8.jpg  
  inflating: semi_test/503a268b.jpg  

In [23]:
test_dl =DataLoader(SiamesetestDataset(np.array(df),path ='/content/semi_test',transform = myvaltransform ),batch_size = 512)

In [24]:
batch_size = 512
import numpy as np
mlen = 10586
testdist = np.zeros(mlen)
model.cuda()
model.eval()
counter = 0
with torch.no_grad():
  for i in tqdm(test_dl):
    img0,img1 = i
    img0,img1 = img0.cuda(),img1.cuda()
    [output1,output2] = model(img0,img1)
    diff = output1-output2
    dist_sq = torch.sum(torch.pow(diff, 2), 1)
    dist = torch.sqrt(dist_sq)
    if(counter+batch_size-1<mlen):
      testdist[counter:counter+batch_size] = dist.cpu()
      counter+=batch_size
    else:
      testdist[counter:] = dist.cpu()
      counter+=batch_size





100%|██████████| 21/21 [04:00<00:00, 11.43s/it]


In [25]:
proba = knn.predict_proba(testdist.reshape(-1,1)).T[1].reshape(-1)

In [26]:
proba

array([1. , 0.6, 0.8, ..., 0.2, 1. , 0.4])

In [27]:
string_list = []
for i in range(10586):
  strn = df["img1_name"][i] +"_"+ df["img2_name"][i]
  string_list.append(strn)

In [28]:
df2 = pd.DataFrame()
df2["id"] = string_list
df2["proba"] = proba

In [30]:
df2.to_csv("60Aucval.csv")

In [31]:
import pickle
with open("/content/drive/MyDrive/ncv/knn.pickle","wb") as file:
  pickle.dump(knn,file)